# Scenario Description 


There exists a lot real-world autonomous driving datasets, such as Waymo and nuPlan.

Those datasets contain rich information about road map, traffic lights, traffic participants and their motions.

Those data can be imported to the MetaDrive so that we can reconstruct an interactive environments and replay (or even alter!) those scenarios recorded in the real world.

To achieve that, we have a separated project called ScenarioNet (https://metadriverse.github.io/scenarionet/) that converts various sources of AD data into a unified data structure, manage those data, and import those data into MetaDrive for creating interactive environments.

As you can see in the next figure of ScenarioNet, the key functionalities developed in ScenarioNet are:

* **Unified Scenario Description**
* **Scenario Data Management**
* **Real-world Scenario Simulation**

 
<img src="figs/scenarionet.png" width="800">


MetaDrive implements the **unified scenario description** in `metadrive/scenario/scenario_description.py` and [ScenarioNet](https://metadriverse.github.io/scenarionet/) implements the data management toolkits. 

For completeness, we will walk through both [the ScenarioNet dataset](scenarionet-dataset), which contains a lots of scenarios, in  and [the Scenario Description](scenario-description), a nested dict that stores the information of one scenario.

## What is the ScenarioNet dataset?
<a id='scenarionet-dataset'></a>

**TL;DR: A established ScenarioNet dataset is a folder containing `dataset_summary.pkl` and `dataset_mapping.pkl` (optionally), along with a set of `.pkl` files each of which stores the unified scenario description of a scenario.**

`dataset_mapping.pkl` contains the mapping from the scenario ID to the relative path. 

`dataset_summary.pkl` summarizes the meta information for each scenario.

A set of `.pkl` files are stored either in the same folder of `dataset_summary.pkl` or in some other folders with the relative path to `dataset_summary.pkl` registered in `dataset_mapping.pkl`. Each of the `.pkl` file stores the serialized scenario descrption, i.e. a python dict, of a scenario.


In this section, we demonstrate how to use the utilities from MetaDrive to easily access scenarios. We prepare two demo datasets, splitting from Waymo Open Dataset and nuScenes Dataset. Here we will demonstrate how to load the files.



You can also find some useful tutorials in ScenarioNet repo: https://github.com/metadriverse/scenarionet/tree/main/tutorial


In [1]:
from metadrive.engine.asset_loader import AssetLoader
import os

waymo_data =  AssetLoader.file_path(AssetLoader.asset_path, "waymo", unix_style=False) # Use the built-in datasets with simulator
os.listdir(waymo_data) # there are 3 waymo scenario files with one 'dataset_summary.pkl' in this example dataset.

['sd_training.tfrecord-00000-of-01000_8a346109094cd5aa.pkl',
 'sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl',
 'dataset_summary.pkl',
 'sd_training.tfrecord-00000-of-01000_c403d5992cab9e0.pkl']

In [2]:
nuscenes_data =  AssetLoader.file_path(AssetLoader.asset_path, "nuscenes", unix_style=False) # Use the built-in datasets with simulator
os.listdir(nuscenes_data) # there are 10 nuscenes scenario file with a 'dataset_summary.pkl' and a 'dataset_summary.pkl'

['nuscenes_6',
 'nuscenes_7',
 'nuscenes_2',
 'nuscenes_5',
 'dataset_mapping.pkl',
 'nuscenes_0',
 'dataset_summary.pkl',
 'nuscenes_1',
 'nuscenes_3',
 'nuscenes_4']

You can find that for the Waymo dataset, as we put all scenarios in the same folder, we don't need the `dataset_mapping.pkl` that routes scenario ID to corresponding `.pkl` file. In the nuScenes dataset, we have both `dataset_mapping.pkl` and `dataset_summary.pkl` because we have a hierachy file structure to store the scenarios.

## How to read a ScenarioNet dataset?


### Read the dataset summary

The first step to read a ScenarioNet dataset is to call the utility function `read_dataset_summary` from MetaDrive.

This function will not read all the data and exhausts your memory as it only read the `dataset_summary.pkl` and optionally the `dataset_mapping.pkl`. The `dataset_summary` contains the summaries (a set of stats) of each scenario in the dataset and the `dataset_mapping` contains the realtive paths of the `.pkl` files of the scenarios relative to the folder that hosts `dataset_summary.pkl` file.

The output of the `read_dataset_summary` is a tuple of three elements:

1) the summary dict mapping from scenario ID to its metadata,
2) the list of all scenarios IDs, and
3) a dict mapping from scenario IDs to the folder that hosts their files.

In [3]:
from metadrive.scenario import utils as sd_utils

dataset_path = waymo_data

print(f"Reading the summary file from Waymo data at: {dataset_path}")

waymo_dataset_summary = sd_utils.read_dataset_summary(dataset_path)
print(f"The dataset summary is a {type(waymo_dataset_summary)}, with lengths {len(waymo_dataset_summary)}.")


Reading the summary file from Waymo data at: /home/zhenghao/metadrive/metadrive/assets/waymo
The dataset summary is a <class 'tuple'>, with lengths 3.


In [4]:
waymo_scenario_summary, waymo_scenario_ids, waymo_scenario_files = waymo_dataset_summary


print(f"The scenario summary is a dict with keys: {waymo_scenario_summary.keys()} \nwhere each value of the dict is the summary of a scenario.\n")

The scenario summary is a dict with keys: dict_keys(['sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl', 'sd_training.tfrecord-00000-of-01000_c403d5992cab9e0.pkl', 'sd_training.tfrecord-00000-of-01000_8a346109094cd5aa.pkl']) 
where each value of the dict is the summary of a scenario.



### What is the summary dict of a scenario?

As you can see, the first return value from the utils will return a dict `waymo_scenario_summary` summarizing all scenarios in the dataset.

Each item in this dict represents the summary of one scenario.

Now let's dive into the detailed struture and see what is included in the summary of one scenario.

In [5]:
example_scenario_summary = waymo_scenario_summary['sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl']
print(f"The summary of a scenario is a dict with keys: {example_scenario_summary.keys()}")

The summary of a scenario is a dict with keys: dict_keys(['coordinate', 'ts', 'metadrive_processed', 'sdc_id', 'dataset', 'scenario_id', 'source_file', 'track_length', 'current_time_index', 'sdc_track_index', 'objects_of_interest', 'tracks_to_predict', 'object_summary', 'number_summary'])


In [6]:
# A string indicating the coordinate system of the scenario.
print(example_scenario_summary["coordinate"])

waymo


In [30]:
# A list containing the wall time of each time step
print(example_scenario_summary["ts"])

[0.0, 0.09997999668121338, 0.19995999336242676, 0.2999500036239624, 0.39994001388549805, 0.49990999698638916, 0.5999400019645691, 0.6999300122261047, 0.7999500036239624, 0.8999699950218201, 0.9999899864196777, 1.0999799966812134, 1.2000000476837158, 1.3000199794769287, 1.4000099897384644, 1.5, 1.5999799966812134, 1.699970006942749, 1.7999399900436401, 1.899940013885498, 1.999959945678711, 2.099950075149536, 2.1999800205230713, 2.299999952316284, 2.4000298976898193, 2.5000200271606445, 2.600029945373535, 2.7000200748443604, 2.8000400066375732, 2.9000399112701416, 3.0000100135803223, 3.099950075149536, 3.199889898300171, 3.299799919128418, 3.3996999263763428, 3.49960994720459, 3.5995099544525146, 3.6994199752807617, 3.799370050430298, 3.899280071258545, 3.9991800785064697, 4.099140167236328, 4.199049949645996, 4.2989702224731445, 4.3988800048828125, 4.49875020980835, 4.598659992218018, 4.6985602378845215, 4.798439979553223, 4.898359775543213, 4.99822998046875, 5.098139762878418, 5.198019

In [8]:
# A boolean indicator saying whether the scenario is exported from MetaDrive.
# Note: we have a replay and record systems in MetaDrive that will generate 
# scenarios with `metadrive_processed=True`.
print(example_scenario_summary["metadrive_processed"])

False


In [9]:
# The object ID of the self-driving car (if exists)
print(example_scenario_summary["sdc_id"])

1629


In [10]:
# Where the scenario comes from
print(example_scenario_summary["dataset"])

waymo


In [11]:
# The ID of the scenario
print(example_scenario_summary["scenario_id"])

2a1e44d405a6833f


In [12]:
# The number of each type of objects. This field is very useful when filtering the datasets.
print(example_scenario_summary["number_summary"])

{'object': 211, 'dynamic_object_states': 8, 'map_features': 358, 'object_types': {'PEDESTRIAN', 'VEHICLE', 'CYCLIST'}, 'object_types_counter': {'VEHICLE': 184, 'PEDESTRIAN': 25, 'CYCLIST': 2}, 'dynamic_object_states_types': {'LANE_STATE_STOP', 'LANE_STATE_UNKNOWN'}, 'dynamic_object_states_counter': {'LANE_STATE_UNKNOWN': 164, 'LANE_STATE_STOP': 564}}


In [13]:
# The summary for each object, including information like the moving distance, type, valid time steps.
print(example_scenario_summary["object_summary"])

{'sdc': {'type': 'VEHICLE', 'object_id': '1629', 'track_length': 91, 'distance': 84.20105910301208, 'valid_length': 91, 'continuous_valid_length': 91}, '325': {'type': 'VEHICLE', 'object_id': '325', 'track_length': 91, 'distance': 2.7487233877182007, 'valid_length': 3, 'continuous_valid_length': 2}, '327': {'type': 'VEHICLE', 'object_id': '327', 'track_length': 91, 'distance': 5.598679721355438, 'valid_length': 15, 'continuous_valid_length': 15}, '332': {'type': 'VEHICLE', 'object_id': '332', 'track_length': 91, 'distance': 103.90164065361023, 'valid_length': 91, 'continuous_valid_length': 91}, '333': {'type': 'VEHICLE', 'object_id': '333', 'track_length': 91, 'distance': 111.29066771268845, 'valid_length': 91, 'continuous_valid_length': 91}, '336': {'type': 'VEHICLE', 'object_id': '336', 'track_length': 91, 'distance': 5.664326503872871, 'valid_length': 23, 'continuous_valid_length': 21}, '337': {'type': 'VEHICLE', 'object_id': '337', 'track_length': 91, 'distance': 0.0, 'valid_length

In [14]:
# You might also find that the summaries of scenarios from different datasets might vary.

print(f"Keys of the scenario from Waymo: {example_scenario_summary.keys()}. \n")
print("Keys of the scenario from nuScene: ", sd_utils.read_dataset_summary(nuscenes_data)[0]['sd_nuscenes_v1.0-mini_scene-0061.pkl'].keys())

Keys of the scenario from Waymo: dict_keys(['coordinate', 'ts', 'metadrive_processed', 'sdc_id', 'dataset', 'scenario_id', 'source_file', 'track_length', 'current_time_index', 'sdc_track_index', 'objects_of_interest', 'tracks_to_predict', 'object_summary', 'number_summary']). 

Keys of the scenario from nuScene:  dict_keys(['dataset', 'metadrive_processed', 'map', 'date', 'coordinate', 'scenario_token', 'id', 'scenario_id', 'sample_rate', 'ts', 'sdc_id', 'object_summary', 'number_summary'])


### What is the other two returns from `read_dataset_summary`?

Recall that `sd_utils.read_dataset_summary(dataset_path)` will return a tuple of three elements. 

The second element is a list containing the `.pkl` file names of all scenarios.

The third element is a dict mapping the `.pkl` file names to their relative path from `dataset_summary.pkl` file.

**Example of Waymo

In [15]:
_, waymo_scenario_ids, waymo_scenario_files = sd_utils.read_dataset_summary(waymo_data)

In [16]:
print(waymo_scenario_ids)

['sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl', 'sd_training.tfrecord-00000-of-01000_c403d5992cab9e0.pkl', 'sd_training.tfrecord-00000-of-01000_8a346109094cd5aa.pkl']


In [17]:
# As those .pkl files are in the same folder of `dataset_summary.pkl`, there is no relative path from pkl to summary file.
waymo_scenario_files

{'sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl': '',
 'sd_training.tfrecord-00000-of-01000_c403d5992cab9e0.pkl': '',
 'sd_training.tfrecord-00000-of-01000_8a346109094cd5aa.pkl': ''}

In [18]:
print("Reading from: ", nuscenes_data)
_, nuscenes_scenario_ids, nuscenes_scenario_files = sd_utils.read_dataset_summary(nuscenes_data)

Reading from:  /home/zhenghao/metadrive/metadrive/assets/nuscenes


In [19]:
nuscenes_scenario_ids

['sd_nuscenes_v1.0-mini_scene-0061.pkl',
 'sd_nuscenes_v1.0-mini_scene-0103.pkl',
 'sd_nuscenes_v1.0-mini_scene-0553.pkl',
 'sd_nuscenes_v1.0-mini_scene-0655.pkl',
 'sd_nuscenes_v1.0-mini_scene-0757.pkl',
 'sd_nuscenes_v1.0-mini_scene-0796.pkl',
 'sd_nuscenes_v1.0-mini_scene-0916.pkl',
 'sd_nuscenes_v1.0-mini_scene-1077.pkl',
 'sd_nuscenes_v1.0-mini_scene-1094.pkl',
 'sd_nuscenes_v1.0-mini_scene-1100.pkl']

In [20]:
# As those .pkl files for nuscenes dataset is stored in separate subfolder from the folder containing `dataset_summary.pkl`,
# there are the relative paths.
#
# For example, the relative path of `sd_nuscenes_v1.0-mini_scene-0061.pkl` can be created by appending:
# nuscenes_scenario_files[`sd_nuscenes_v1.0-mini_scene-0061.pkl`] + '/' + 'sd_nuscenes_v1.0-mini_scene-0061.pkl'

nuscenes_scenario_files

{'sd_nuscenes_v1.0-mini_scene-0061.pkl': 'nuscenes_0',
 'sd_nuscenes_v1.0-mini_scene-0103.pkl': 'nuscenes_1',
 'sd_nuscenes_v1.0-mini_scene-0553.pkl': 'nuscenes_2',
 'sd_nuscenes_v1.0-mini_scene-0655.pkl': 'nuscenes_3',
 'sd_nuscenes_v1.0-mini_scene-0757.pkl': 'nuscenes_4',
 'sd_nuscenes_v1.0-mini_scene-0796.pkl': 'nuscenes_5',
 'sd_nuscenes_v1.0-mini_scene-0916.pkl': 'nuscenes_6',
 'sd_nuscenes_v1.0-mini_scene-1077.pkl': 'nuscenes_7',
 'sd_nuscenes_v1.0-mini_scene-1094.pkl': 'nuscenes_7',
 'sd_nuscenes_v1.0-mini_scene-1100.pkl': 'nuscenes_7'}

## What is the Scenario Description? ⭐
<a id='scenario-description'></a>

**TL;DR: Scenario Description is a nested python dict that stores the information of one AD scenario.**

By defining the unified data structure, from the input end, we can define a set of toolkits to convert each dataset into the dataset in the form of Scenario Description.

Then we can store each scenario into a `.pkl` file in the local file system and manage those files with [ScenarioNet](https://github.com/metadriverse/scenarionet).

From the output end, no matter where the source data is from, we can reconstruct interactive environment via MetaDrive and conduct downstream tasks like RL training by reading from the dataset containing those `.pkl` files.

Here is an example of what the Scenario Description looks like:


```python
scenario = {

    # ===== Meta data about the scenario =====
    # string. The name of the scenario
    "id": "Waymo-001",

    # string. The version of data format.
    "version": "MetaDrive v0.3.0.1",

    # int. The length of all trajectory and state arrays (T).
    "length": 200,

    # ===== Meta data ===
    "metadata": {

        # np.ndarray in (T, ). The time stamp of each time step.
        "ts": np.array([0.0, 0.1, 0.2, ...], dtype=np.float32),


        # bool. Whether the scenario is processed and exported by MetaDrive.
        # Some operations may be done, such as interpolating the lane to
        # make way points uniformly scattered in given interval.
        "metadrive_processed": True,

        # string. Coordinate system.
        "coordinate": "metadrive",

        # optional keys
        "source_file": "training_20s.tfrecord-00014-of-01000",
        "dataset": "waymo",
        "scenario_id": "dd0c8c27fdd6ef59",  # Used in Waymo dataset
        "seed": 512,
        "history_metadata": {},

        "sdc_id": "172",  # A key exists in tracks

    },

    # ===== Trajectories of active participants, e.g. vehicles, pedestrians =====
    # a dict mapping object ID to it's state dict.
    "tracks": {
        "vehicle1": {

            # The type string in metadrive.scenario.MetaDriveType
            "type": "VEHICLE",

            # The state dict. All values must have T elements.
            "state": {
                "position": np.ones([200, 3], dtype=np.float32),
                ...
            },

            # The meta data dict. Store useful information about the object. type in metadata could be those from
            # different dataset
            "metadata": {
                "type": "VEHICLE",
                "track_length": 200,
                "object_id": "vehicle1",

                # Optional keys
                "agent_name": "default_agent",
                "policy_spawn_info": {  # Information needed to re-instantiate the policy
                    "policy_class": ("metadrive.policy.idm_policy", "IDMPolicy"),
                    "args": ...,
                    "kwargs": ...,
                }
            }
        },

        "pedestrian1": ...
    },

    # ===== States sequence of dynamics objects, e.g. traffic light =====
    # a dict mapping object ID to it's state dict.
    "dynamic_map_states": {
        "trafficlight1": {

            # The type string in metadrive.scenario.MetaDriveType
            "type": "TRAFFIC_LIGHT",

            # The state dict. All values must have T elements.
            "state": {
                "object_state": np.ones([200, ], dtype=int),
                ...
            },

            # The meta data dict. Store useful information about the object
            "metadata": {
                "type": "TRAFFIC_LIGHT",
                "track_length": 200,
            }
    }

    # ===== Map features =====
    # A dict mapping from map feature ID to a line segment
    "map_features": {
        "219": {
            "type": "LANE_SURFACE_STREET",
            "polyline": np.array in [21, 2],  # A set of 2D points describing a line segment
            # optional, only works for lane
            "polygon": np.array in [N, 2] # A set of 2D points representing convexhull
        },
        "182": ...
        ...
    }
}

# Note: Example is from the docstring of metadrive/scenario/scenario_description.py
```

### Read a scenario description in `.pkl` with MetaDrive utils

We want to avoid using any advanced tools to ensure the maximal compatibility. Here let me demonstrate how to read a `.pkl` file and instantiate a Scenario Descrption from it.

We will use `read_scenario` to read a given `.pkl` file, and it will return a `ScenarioDescrption` instance. A `ScenarioDescrption` is a wrapper of native python dict with some useful utility functions.

In [21]:
# Get the dataset path
dataset_path = waymo_data
print("Dataset path: ", dataset_path)

# Get the scenario .pkl file name
_, scenario_ids, dataset_mapping = sd_utils.read_dataset_summary(dataset_path)
# Just pick the first scenario
scenario_pkl_file = scenario_ids[0]

# Get the relative path to the .pkl file
print("The pkl file relative path: ", dataset_mapping[scenario_pkl_file])  # A empty path

# Get the absolute path to the .pkl file
abs_path_to_pkl_file = os.path.join(dataset_path, dataset_mapping[scenario_pkl_file], scenario_pkl_file)
print("The pkl file absolute path: ", abs_path_to_pkl_file)

# Call utility function in MD and get the ScenarioDescrption object
scenario = sd_utils.read_scenario_data(abs_path_to_pkl_file)

print(f"\nThe raw data type after reading the .pkl file is {type(scenario)}")

Dataset path:  /home/zhenghao/metadrive/metadrive/assets/waymo
The pkl file relative path:  
The pkl file absolute path:  /home/zhenghao/metadrive/metadrive/assets/waymo/sd_training.tfrecord-00000-of-01000_2a1e44d405a6833f.pkl

The raw data type after reading the .pkl file is <class 'metadrive.scenario.scenario_description.ScenarioDescription'>


In [22]:
print(f"The keys in a ScenarioDescrption are: {scenario.keys()}")

The keys in a ScenarioDescrption are: dict_keys(['id', 'version', 'length', 'tracks', 'dynamic_map_states', 'map_features', 'metadata'])


In [43]:
# Now you can play with the data in scenario as it's like a python dict:

print(f"The SDC vehicle name is: {scenario['metadata']['sdc_id']}")

sdc_id = scenario['metadata']['sdc_id']
print(f"The SDC vehicle type is: {scenario['tracks'][sdc_id]['type']}")

print(f"The SDC vehicle position at last time step is: {scenario['tracks'][sdc_id]['state']['position'][-1]}")

print(f"There are traffic lights: {scenario['dynamic_map_states'].keys()}")

print(f"The traffic light 316's state is: {scenario['dynamic_map_states']['316']['state']['object_state']}")


The SDC vehicle name is: 1629
The SDC vehicle type is: VEHICLE
The SDC vehicle position at last time step is: [ 2976.9563  -3251.819     186.91092]
There are traffic lights: dict_keys(['226', '227', '272', '316', '317', '321', '354', '355'])
The traffic light 316's state is: ['LANE_STATE_UNKNOWN', 'LANE_STATE_UNKNOWN', 'LANE_STATE_UNKNOWN', 'LANE_STATE_UNKNOWN', 'LANE_STATE_UNKNOWN', 'LANE_STATE_UNKNOWN', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'LANE_STATE_STOP', 'L

### Useful utility functions from ScenarioDescrption

Note that these functions are class functions.

In [23]:
# Check the self-consistency of a scenario and whether the required fields are filled.
scenario.sanity_check?
scenario.sanity_check(scenario)  # If no error is raised, the check passes.

Signature:
scenario.sanity_check(
    scenario_dict,
    check_self_type=False,
    valid_check=False,
)
Docstring:
Check if the input scenario dict is self-consistent and has filled required fields.

The required high-level fields include tracks, dynamic_map_states, metadata, map_features.
For each object, the tracks[obj_id] should at least contain type, state, metadata.
For each object, the tracks[obj_id]['state'] should at least contain position, heading.
For each lane in map_features, map_feature[map_feat_id] should at least contain polyline.
For metadata, it should at least contain metadrive_processed, coordinate and timestep.
We have more checks to ensure the consistency of the data.

Args:
    scenario_dict: the input dict.
    check_self_type: if True, assert the input dict is a native Python dict.
    valid_check: if True, we will assert the values for a given timestep are zeros if valid=False at that
        timestep.
File:      ~/metadrive/metadrive/scenario/scenario_descrip

In [24]:
# Convert the ScenarioDescrption to a python dict.
scenario_dict = scenario.to_dict()
print(f"The keys in the converted dict are: {scenario_dict.keys()}")

The keys in the converted dict are: dict_keys(['id', 'version', 'length', 'tracks', 'dynamic_map_states', 'map_features', 'metadata'])


In [25]:
# Return the object info dict of the SDC.
scenario.get_sdc_track()

{'type': 'VEHICLE',
 'state': {'position': array([[ 2935.023  , -3324.672  ,   187.30748],
         [ 2935.801  , -3323.4658 ,   187.30122],
         [ 2936.5781 , -3322.262  ,   187.28738],
         [ 2937.3518 , -3321.0627 ,   187.28615],
         [ 2938.1191 , -3319.8745 ,   187.28394],
         [ 2938.878  , -3318.6953 ,   187.27779],
         [ 2939.6355 , -3317.5208 ,   187.26611],
         [ 2940.3896 , -3316.351  ,   187.25655],
         [ 2941.139  , -3315.1902 ,   187.2451 ],
         [ 2941.8835 , -3314.0383 ,   187.23409],
         [ 2942.6218 , -3312.897  ,   187.22908],
         [ 2943.3516 , -3311.7654 ,   187.22733],
         [ 2944.078  , -3310.641  ,   187.2228 ],
         [ 2944.8005 , -3309.526  ,   187.2146 ],
         [ 2945.5159 , -3308.4202 ,   187.20764],
         [ 2946.2234 , -3307.3245 ,   187.20921],
         [ 2946.9236 , -3306.2424 ,   187.20998],
         [ 2947.6172 , -3305.1675 ,   187.203  ],
         [ 2948.3066 , -3304.101  ,   187.19052],
         

In [26]:
# Compute the moving distance of SDC.
sd_utils.ScenarioDescription.sdc_moving_dist(scenario)

84.20105910301208

In [27]:
# Verify if the .pkl file is valid.
sd_utils.ScenarioDescription.is_scenario_file(abs_path_to_pkl_file)

True

In [28]:
# Return the summary of an object. Here we use the SDC.
sd_utils.ScenarioDescription.get_object_summary(
    object_dict=scenario.get_sdc_track(), 
    object_id=scenario['metadata']['sdc_id']
)

{'type': 'VEHICLE',
 'object_id': '1629',
 'track_length': 91,
 'moving_distance': 84.20105910301208,
 'valid_length': 91,
 'continuous_valid_length': 91}

In [46]:
scenario['metadata']['source_file']

'training.tfrecord-00000-of-01000'

In [45]:
scenario['metadata']['object_summary']

{'sdc': {'type': 'VEHICLE',
  'object_id': '1629',
  'track_length': 91,
  'distance': 84.20105910301208,
  'valid_length': 91,
  'continuous_valid_length': 91},
 '325': {'type': 'VEHICLE',
  'object_id': '325',
  'track_length': 91,
  'distance': 2.7487233877182007,
  'valid_length': 3,
  'continuous_valid_length': 2},
 '327': {'type': 'VEHICLE',
  'object_id': '327',
  'track_length': 91,
  'distance': 5.598679721355438,
  'valid_length': 15,
  'continuous_valid_length': 15},
 '332': {'type': 'VEHICLE',
  'object_id': '332',
  'track_length': 91,
  'distance': 103.90164065361023,
  'valid_length': 91,
  'continuous_valid_length': 91},
 '333': {'type': 'VEHICLE',
  'object_id': '333',
  'track_length': 91,
  'distance': 111.29066771268845,
  'valid_length': 91,
  'continuous_valid_length': 91},
 '336': {'type': 'VEHICLE',
  'object_id': '336',
  'track_length': 91,
  'distance': 5.664326503872871,
  'valid_length': 23,
  'continuous_valid_length': 21},
 '337': {'type': 'VEHICLE',
  'o

In [44]:
sd_utils.ScenarioDescription.get_number_summary(
    scenario=scenario
)

KeyError: 'moving_distance'

In [ ]:
scenario['metadata']['object_summary']

In [67]:
waymo_data

'/home/zhenghao/metadrive/metadrive/assets/waymo'

In [51]:
import pickle
with open(waymo_data + '/dataset_summary.pkl', 'rb') as f:
    dataset_summary = pickle.load(f)

In [63]:
new_dataset_summary = {}

def _replace(d):
    d['moving_distance'] = d.pop('distance')
    return d

for sid, sinfo in dataset_summary.items():
    sinfo['object_summary'] = {
        vid: _replace(vstat)
        for vid, vstat in sinfo['object_summary'].items()
    }
    new_dataset_summary[sid] = sinfo

In [66]:
with open(waymo_data + '/dataset_summary.pkl', 'wb') as f:
    pickle.dump(new_dataset_summary, f)